In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.models import Model
import glob
from keras.models import Sequential
from keras.layers import *
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
remote = callbacks.RemoteMonitor(root='http://localhost:9000')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from tqdm import tqdm_notebook as tq
from random import shuffle
import os
from keras.utils.visualize_util import plot
from keras.optimizers import *
from shutil import copyfile

%matplotlib inline


Using Theano backend.
Using gpu device 0: GeForce GTX 960M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5105)


We need to move boats around.

In [2]:
def group_boats(groups, new_folder = "./boats_id_train/"):
    files_list = glob.glob('./boats_id/*/*/*.jpg')
    tuples = [x.split("\\") for x in files_list]
    if not os.path.exists(new_folder):
        os.makedirs(new_folder)
        
    for i in range(len(groups) + 1):
        if not os.path.exists(new_folder + str(i) + "/"):
            os.makedirs(new_folder + str(i) + "/")
    
    moved = [False for _ in tuples]
    
    for i, group in enumerate(tq(groups)):
        if type(group) is int:
            group  = [group]
        
        for folder_idx in group:
            for j, tup in enumerate(tuples):
                if int(tup[2]) == folder_idx:
                    copyfile(files_list[j], new_folder + str(i + 1) + "/" + tup[-1])
                    moved[j] = True
    
    for j, move in enumerate(tq(moved)):
        if not move:
            copyfile(files_list[j], new_folder + "0" + "/" + tuples[j][-1])
    

In [23]:
group_boats([[0,1],[2,3],4,5,6,[7,8],[9,10],11,12,13,[14,15],20,22,36,26,27,32,42])

Now we need to have the same number of pictures in each class. So we'll add new ones.

In [3]:
def split_train_test(nb_images, train_dir):
    temp = train_dir.find("train")
    test_dir = train_dir[:temp] + "test" + train_dir[temp+5:]
    
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)
        
    classes = os.listdir(train_dir)
        
    train_dir = [train_dir + x + "/" for x in classes]
    test_dir = [test_dir + x + "/" for x in classes]
    
    for folder in test_dir:
        if not os.path.exists(folder):
            os.makedirs(folder)
    
    for train_class, test_class in zip(train_dir, test_dir):
        files = os.listdir(train_class)[:nb_images]
        for file in files:
            copyfile(train_class + file,test_class + file)
            os.remove(train_class + file)

In [5]:
#split_train_test(5,"boats_id_train/")

In [8]:
def create_new_pictures(train_folder, class_folder, generator,total_images, 
                        batch_size = 32, target_size = (256,256)):
    count = len(os.listdir(train_folder + "/" + class_folder))
    #count = 0  
    gen_folder = train_folder + "_gen/"
        
    if not os.path.exists(gen_folder):
        os.makedirs(gen_folder)
        
    if not os.path.exists(gen_folder + class_folder):
        os.makedirs(gen_folder + class_folder)
        
    
    progbar = tq(total = total_images, leave=False)
    progbar.update(count)
    
    
    for X in generator.flow_from_directory(train_folder, target_size=target_size,
                                             classes=[class_folder], 
                                             save_to_dir=train_folder + "_gen/" + class_folder, 
                                             batch_size=batch_size, color_mode="rgb"):
        count +=batch_size
        progbar.update(batch_size)
        if count >= total_images:
            break
    progbar.close()
    

In [11]:
my_generator = ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=2,
    width_shift_range=0.01,
    height_shift_range=0.01,
    shear_range=0.001,
    zoom_range=0.,
    channel_shift_range=0.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    noise = 30)

In [12]:
create_new_pictures("boats_id_train", "4", my_generator,50, 8, (None,None))

Found 400 images belonging to 1 classes.


In [ ]:
for string in os.listdir("boats_id_train"):
    create_new_pictures("boats_id_train", string, my_generator,500, 32, (128,128))

In [7]:
def equilibrate_dataset(folder, nb_images_per_class, generator):
    for string in os.listdir(folder):
        create_new_pictures(folder, string, generator,nb_images_per_class, 32, (64,64))

In [9]:
equilibrate_dataset("boats_id_train - Copy", 500,my_generator)

Found 1117 images belonging to 1 classes.
Found 856 images belonging to 1 classes.
Found 31 images belonging to 1 classes.
Found 34 images belonging to 1 classes.
Found 31 images belonging to 1 classes.
Found 22 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 9 images belonging to 1 classes.
Found 51 images belonging to 1 classes.
Found 54 images belonging to 1 classes.
Found 83 images belonging to 1 classes.
Found 214 images belonging to 1 classes.
Found 46 images belonging to 1 classes.
Found 88 images belonging to 1 classes.
Found 235 images belonging to 1 classes.
Found 542 images belonging to 1 classes.
Found 39 images belonging to 1 classes.
Found 245 images belonging to 1 classes.
